## Imports

In [ ]:
import cupy as cp
from tifffile import tifffile 
from pathlib import Path
from sparse_recon.sparse_deconv import sparse_deconv
from cucim.skimage.exposure import match_histograms
import napari

## Load data and normalize histograms through z-stack

Channel 1

In [ ]:
img_WF_C1_path = Path(r'C:\Users\qi2lab\Documents\sim\C1-widefield-1.tif')
img_WF_C1 = tifffile.imread(img_WF_C1_path)
img_WF_C1 = img_WF_C1[5:-1,:,:]

img_SIM_C1_path = Path(r'C:\Users\qi2lab\Documents\sim\C1-sim_sr-1.tif')
img_SIM_C1 = tifffile.imread(img_SIM_C1_path)
img_SIM_C1 = img_SIM_C1[5:-1,:,:]
img_SIM_C1[img_SIM_C1<0]=0
scale_C1 = img_SIM_C1.max()
img_SIM_C1 = img_SIM_C1/scale_C1
#img_SIM_C1 = attenuation_transform.preprocess(img_SIM_C1)
cp_img_SIM_C1 = cp.asarray(img_SIM_C1)
z_slices = img_SIM_C1.shape[0]
for z_idx in cp.arange(0,z_slices):
    cp_img_SIM_C1[z_idx,:,:] = match_histograms(cp_img_SIM_C1[z_idx,:,:],cp_img_SIM_C1[z_slices//2,:,:])
#img_SIM_C1 = cp.asnumpy(equalize_adapthist(cp.asarray(img_SIM_C1),nbins=2**16))
img_SIM_C1 = scale_C1 * cp.asnumpy(cp_img_SIM_C1)

cp.clear_memo()


Channel 2

In [ ]:
img_WF_C2_path = Path(r'C:\Users\qi2lab\Documents\sim\C2-widefield-1.tif')
img_WF_C2 = tifffile.imread(img_WF_C2_path)
img_WF_C2 = img_WF_C2[5:-1,:,:]

img_SIM_C2_path = Path(r'C:\Users\qi2lab\Documents\sim\C2-sim_sr-1.tif')
img_SIM_C2 = tifffile.imread(img_SIM_C2_path)
img_SIM_C2 = img_SIM_C2[5:-1,:,:]
img_SIM_C2[img_SIM_C2<0]=0
scale_C2 = img_SIM_C2.max()
img_SIM_C2 = img_SIM_C2/scale_C2
#img_SIM_C2 = attenuation_transform.preprocess(img_SIM_C2)
cp_img_SIM_C2 = cp.asarray(img_SIM_C2)
z_slices = img_SIM_C1.shape[0]
for z_idx in cp.arange(0,z_slices):
    cp_img_SIM_C2[z_idx,:,:] = match_histograms(cp_img_SIM_C2[z_idx,:,:],cp_img_SIM_C2[z_slices//2,:,:])
#img_SIM_C1 = cp.asnumpy(equalize_adapthist(cp.asarray(img_SIM_C1),nbins=2**16))
img_SIM_C2 = scale_C2 * cp.asnumpy(cp_img_SIM_C2)

cp.clear_memo()

## Perform sparse deconvolution

In [ ]:
img_SSIM_t01_C1 = sparse_deconv(img_SIM_C1, [.130/ .0325, .130/.0325],prior=4,sparse_iter=100,tcontinuity=0.2,fidelity=50,sparsity=5,deconv_type=3,up_sample=0)
img_SSIM_t01_C2 = sparse_deconv(img_SIM_C2, [.165/ .0325, .165/.0325],prior=4, sparse_iter=100,tcontinuity=0.2,fidelity=50,sparsity=5,deconv_type=3,up_sample=0)

## View widefield, 2D-SIM w/ optical sectioning, and Sparse deconvolution of 2D-SIM in Napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(img_WF_C2,scale=[.230, 0.065,.065],colormap='bop orange')
viewer.add_image(img_SIM_C2,scale=[.230, 0.0325,.0325],colormap='bop orange',blending='additive')
viewer.add_image(img_SSIM_t01_C2,scale=[.230, 0.0325,.0325],colormap='bop orange',blending='additive')

viewer.add_image(img_WF_C1,scale=[.230, 0.065,.065],colormap='bop blue')
viewer.add_image(img_SIM_C1,scale=[.230, 0.0325,.0325],colormap='bop blue',blending='additive')
viewer.add_image(img_SSIM_t01_C1,scale=[.230, 0.0325,.0325],colormap='bop blue',blending='additive')
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'